## Fastai Image classification Wild Cats


In [7]:
from fastai.vision import *
import fastai; fastai.__version__
import numpy as np

**Let's see how fastai handles MNIST**

In [8]:
# path = untar_data(URLs.MNIST_SAMPLE)
# data = ImageDataBunch.from_folder(path)
# learn = cnn_learner(data, models.resnet18, metrics=accuracy)
# learn.fit(1)

**Classify images of brown bears, panda bears, tigers, lions, and leopards**

In [9]:
# We can verify that we don’t have any corrupt images using the verify_images method.
import os
path = Path('../input/cats-version-3/cats')
for folder in ('cheetahs', 'jaguars', 'leopards','panthers','tigers'):
    print(folder)
    verify_images(os.path.join(path, folder), delete=True, max_size=700)

cheetahs


jaguars


leopards


[Errno 30] Read-only file system: '../input/cats-version-3/cats/leopards/snow-leopard-coloring-page.jpg'
[Errno 30] Read-only file system: '../input/cats-version-3/cats/leopards/images (3).png'
[Errno 30] Read-only file system: '../input/cats-version-3/cats/leopards/NEW-GALLERY-LEOPARD-THUMBNAIL-960x539.jpg'
[Errno 30] Read-only file system: '../input/cats-version-3/cats/leopards/Snow_leopard_(Panthera_uncia)_Natural_History_Museum_London_Mammals_Gallery_04.jpg'
[Errno 30] Read-only file system: '../input/cats-version-3/cats/leopards/images (1).png'
[Errno 30] Read-only file system: '../input/cats-version-3/cats/leopards/Snow-Leopard-Cub-Signed-Poster-by-Ian-Coleman-1-820x820.jpg'
[Errno 30] Read-only file system: '../input/cats-version-3/cats/leopards/Leopard-2.png'
[Errno 30] Read-only file system: '../input/cats-version-3/cats/leopards/jA3IszD.jpg'
[Errno 30] Read-only file system: '../input/cats-version-3/cats/leopards/images (2).png'


OSError: [Errno 30] Read-only file system: '../input/cats-version-3/cats/leopards/snow-leopard-coloring-page.jpg'

In [ ]:

np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=path, valid_pct=0.33,
                                  ds_tfms=get_transforms(),bs=15 ,size=500, num_workers=0).normalize(imagenet_stats)
# Setting num_workers=0 prevents crushing!

In [ ]:
data.classes

In [ ]:
data.show_batch(rows=5, figsize=(7, 8))

In [ ]:
from fastai.metrics import error_rate # 1 - accuracy
learn = create_cnn(data, models.resnet34, metrics=error_rate)
# The created model uses the resnet34 architecture, with weights pretrained on the imagenet dataset.
# By default, only the fully connected layers at the top are unfrozen (can be trained), 
# which if you are familiar with transfer learning makes perfect sense.

In [ ]:
# defaults.device = torch.device('cuda') # makes sure the gpu is used
learn.fit_one_cycle(8)

In [ ]:
# Will take too long without GPU
learn.model_dir='/kaggle/working/'
learn.unfreeze() # must be done before calling lr_find
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, max_lr=slice(3e-5, 3e-4))

In [ ]:
learn.save('..animal-detection-stage-1')

## Interpretation

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(6, figsize=(15,15))
# Show images in `top_losses` along with their prediction, actual, loss, and probability of actual class.

In [ ]:
learn.summary()